In [1]:
%matplotlib inline

# 优化模型参数

现在我们有了模型和数据，是时候通过优化数据上的参数来训练、验证和测试我们的模型了。 训练模型是一个迭代过程； 在每次迭代中（称为 *epoch*），模型对输出进行猜测，计算其猜测中的误差（*loss*），收集误差相对于其参数的导数（如我们在模块中看到的） ，并使用梯度下降**优化**这些参数。 有关此过程的更详细演练，请查看 [来自 3Blue1Brown 的反向传播](https://www.youtube.com/watch?v=tIeHLnjs5U8) 上的视频。

##先决条件代码

我们在 **Datasets & DataLoaders** 和 **Build Model** 上加载前面模块的代码



In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 设置超参数

超参数是可调节的参数，可让您控制模型优化过程。
不同的超参数值会影响模型训练和收敛速度
（[阅读更多]（https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html）关于超参数调整）

我们为训练定义了以下超参数：
  - **Number of Epochs** - 迭代数据集的次数
  - **Batch Size** - 模型在每个 epoch 中看到的数据样本数量
  - **Learning Rate** - 在每个批次/时期更新模型参数的程度。 较小的值会导致学习速度变慢，而较大的值可能会导致训练过程中出现不可预测的行为。

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 添加优化循环

一旦我们设置了我们的超参数，我们就可以使用优化循环来训练和优化我们的模型。每个
优化循环的迭代称为 **epoch**。

每个时代由两个主要部分组成：
 - **The Train Loop** - 迭代训练数据集并尝试收敛到最佳参数。
 - **The Validation/Test Loop** - 迭代测试数据集以检查模型性能是否正在提高。

让我们简要地熟悉一下训练循环中使用的一些概念。跳到
请参阅优化循环的“full-impl-label”。

### 添加损失函数

当提供一些训练数据时，我们未经训练的网络可能无法给出正确的
回答。 **损失函数**衡量得到的结果与目标值的不同程度，
它是我们在训练过程中想要最小化的损失函数。为了计算损失，我们做了一个
使用我们给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括用于回归任务的 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)（均方误差），以及
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（负对数似然）进行分类。
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) 结合了`nn.LogSoftmax` 和`nn.NLLLoss`。

我们将模型的输出 logits 传递给 `nn.CrossEntropyLoss`，它将对 logits 进行归一化并计算预测误差。

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

优化是在每个训练步骤中调整模型参数以减少模型误差的过程。 **优化算法**定义了这个过程是如何执行的（在这个例子中我们使用随机梯度下降）。
所有优化逻辑都封装在 ``optimizer`` 对象中。 在这里，我们使用 SGD 优化器； 此外，还有许多[不同的优化器](https://pytorch.org/docs/stable/optim.html)
在 PyTorch 中可用，例如 ADAM 和 RMSProp，它们更适用于不同类型的模型和数据。

我们通过注册模型需要训练的参数并传入学习率超参数来初始化优化器。



In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分三步进行：
  * 调用`optimizer.zero_grad()` 来重置模型参数的梯度。 默认情况下渐变相加； 为了防止重复计算，我们在每次迭代时明确地将它们归零。
  * 通过调用`loss.backwards()` 来反向传播预测损失。 PyTorch 将损失的梯度存入 w.r.t. 每个参数。
  * 一旦我们有了梯度，我们就调用 ``optimizer.step()`` 来通过向后传递中收集的梯度来调整参数。

## 完整实现

我们定义了循环优化代码的 `train_loop`，以及循环优化代码的 `test_loop`
根据我们的测试数据评估模型的性能。

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化损失函数和优化器，并将其传递给 `train_loop` 和 `test_loop`。
随意增加 epochs 的数量以跟踪模型的改进性能。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302778  [    0/60000]
loss: 2.297335  [ 6400/60000]
loss: 2.283023  [12800/60000]
loss: 2.272815  [19200/60000]
loss: 2.286411  [25600/60000]
loss: 2.254095  [32000/60000]
loss: 2.261106  [38400/60000]
loss: 2.249511  [44800/60000]
loss: 2.233091  [51200/60000]
loss: 2.205864  [57600/60000]
Test Error: 
 Accuracy: 43.4%, Avg loss: 0.034827 

Epoch 2
-------------------------------
loss: 2.225592  [    0/60000]
loss: 2.239306  [ 6400/60000]
loss: 2.188773  [12800/60000]
loss: 2.178016  [19200/60000]
loss: 2.227735  [25600/60000]
loss: 2.153194  [32000/60000]
loss: 2.171523  [38400/60000]
loss: 2.147953  [44800/60000]
loss: 2.107409  [51200/60000]
loss: 2.067630  [57600/60000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 0.032724 

Epoch 3
-------------------------------
loss: 2.098490  [    0/60000]
loss: 2.124602  [ 6400/60000]
loss: 2.026889  [12800/60000]
loss: 2.017247  [19200/60000]
loss: 2.079707  [25600/60000]
loss: 1.978768  [32000/600

您可能已经注意到模型最初并不是很好（没关系！）。 尝试运行循环以获得更多的“epochs”或将“learning_rate”调整为更大的数字。 也可能是我们选择的模型配置可能不是此类问题的最佳配置（事实并非如此）。 后面的课程将深入研究适用于视力问题的模型形状。
